# Recurrent Neural Network for Wind Speed Prediction

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time

## DATA
Your data is saved in a `.mat` file called <code>hot_wire_data</code> in the usual directory (<code>''/resource/asnlib/publicdata</code>). It consists of a mutlidimensional time series of measurements saved as variables. 

The file contains all measurement parameters but only two are needed: `U` - the measured speed at two locations and `samp_rate` - frequency of measurements which will be essential in creating a time variable. We will be working with the second location (second column of `U`).

In [4]:
import h5py
with h5py.File('D:/GitHub/RNN_Wind_Speed_Predict/resource/asnlib/publicdata/hot_wire_data.mat', 'r') as f:
    U = list(f['U'])
    freq = list(f['samp_rate'])
    
    #显示文件中所有变量的名称，可以了解数据的结构
    print(list(f.keys()))

['AtmP', 'KinVisc', 'TempK', 'U', 'Uinfty', 'Utau', 'd1', 'd2', 'd99', 'dH', 'samp_rate', 'x_since_tripping', 'y']


Create datasets for training and testing. Use the frequency to create a variable of time elapsed during the measurement, expressed in seconds. Make sure you only use the second column of `U` for your predictions (only the second location). To prevent kernel timeout, we are only going to be working with the first 10 000 time steps.

In [6]:
def prep_dataset(U, freq, split_ratio):
    ###
    ### YOUR CODE HERE
    
    #Convert U to np array and store in X
    X = np.array(U)
    #行列转换
    X = X.T
    #Take second column of U
    X = X[:, 1]   
    #上面操作将X编程一维的了，这里将一维张量变成二维
    X=X.reshape(-1, 1)
    #题目要求只用前10000 time step
    X = X[:10000]
    

    #Create time array
    time = np.arange(len(X)) / freq
    #行列转换
    time = time.T
    #题目要求只用前10000 time step
    time = time[:10000]
    
    
    #计算train和validation set的数量
    n_train = int(len(X)* split_ratio)
    n_val = int(len(X)* round(1.0-split_ratio, 3))#round因为python这里bug算出来不是整数 
    
    
    # Splitting
    x_train, time_train = X[:n_train], time[:n_train]
    x_valid, time_valid = X[n_train:n_train+n_val], time[n_train:n_train+n_val]

    ###
    return time_train, x_train, time_valid, x_valid
split_ratio = 0.8
time_train, x_train, time_valid, x_valid = prep_dataset(U,freq,split_ratio)



In [7]:
n = 2
assert time_train[n] == n/freq[0],"Make sure you convert your frequency to time elapsed"
assert len(time_train)/len(time_valid) == 4,"Make sure you split your time elapsed as well as the data itself"
assert len(x_train)/len(x_valid) == 4,"Make sure 80% of your data is in the training set"
assert len(x_train) + len(x_valid) == 10000, "Include only the first 10 000 time steps in your datasets"

### Windowing

In [8]:
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    ###
    ### YOUR CODE HERE
    
    # 创建TensorFlow的数据集
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # 将数据集分割成指定长度的窗口，每个窗口长度为 window_size + 1
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    # 将窗口展平为张量
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # 将每个窗口分为输入和标签（最后一个值作为标签，前面的作为输入特征）
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    # 打乱数据集以防止模型过拟合
    dataset = dataset.shuffle(buffer_size=max(1, shuffle_buffer))
    # 分批次（batch）处理
    dataset = dataset.batch(batch_size).prefetch(1)

    ###
    return dataset

# Create training and test sets
# Define your window_size, batch_size and shuffle_buffer_size below
###
### YOUR CODE HERE

window_size = 50
batch_size = 32
shuffle_buffer_size = 100

valid_dataset = windowed_dataset(x_valid, 
                            window_size, 
                            batch_size, 
                            shuffle_buffer=shuffle_buffer_size)

###
train_dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
test_dataset = windowed_dataset(x_valid, window_size, batch_size, 1)


## MODEL

In [10]:
def build_model():
    ###
    ### YOUR CODE HERE
    
    
    #RNN和LSTM层的单元数（隐藏节点数）
    n_units=32
    
    model = tf.keras.models.Sequential([
                    # The Lambda layer adds an extra dimension to the input to 
                    # match the expected input shape to RNN layers.
                    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis = -1), 
                                           input_shape = [None]),
        
                    # These 4 layers are the RNN model. 
                    #LSTM层
                    tf.keras.layers.LSTM(units=n_units, 
                                         return_sequences = True), # Required for shape compatibility with SimpleRNN
        
                    # These 4 layers are the RNN model. 
                    #LSTM层
                    tf.keras.layers.LSTM(units=n_units, 
                                         return_sequences = True), # Required for shape compatibility with SimpleRNN
        
                
        
                    # SimpleRNN 层，用于捕获短期依赖性
                    tf.keras.layers.SimpleRNN(units=n_units), 
        
                    # Connect the RNN model to the output label (size 1)
                    #全连接层
                    tf.keras.layers.Dense(units=1),
        
                    # This last Lambda layer scales the output by 100 to match 
                    # the target scale, which is often useful in regression problems.  
                    tf.keras.layers.Lambda(lambda x: x * 20.0) 
                    ])
    
    model.compile(optimizer = tf.keras.optimizers.Adam(),#使用Adam优化器
                  loss = tf.keras.losses.Huber(),#使用Huber损失
                  metrics = ['mae', 'mape'])#评价指标使用平均绝对差（mae）
    
    ###
    return model
model = build_model()
model.summary()

C:\Users\jason\anaconda3\Lib\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lambda (Lambda)                      │ (None, None, 1)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, None, 32)            │           4,352 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, None, 32)            │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn (SimpleRNN)               │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │              33 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda_1 (Lambda)                    │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,785 (57.75 KB)

 Trainable params: 14,785 (57.75 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
for example in train_dataset:
    assert example[0].numpy().shape[1] == window_size
    assert example[0].numpy().shape[0] <= batch_size
assert type(window_size)==int and type(batch_size) == int and type(shuffle_buffer_size)==int, "Define the 3 integer variables for windowing"

## TRAIN

In [13]:
def train_model(train_dataset):
    start_time = time.time()
    ###
    ### YOUR CODE HERE
    
    
    history = model.fit(train_dataset, 
                        epochs = 30, validation_data=valid_dataset)
    
    
    ###
    end_time = time.time()
    runtime = end_time-start_time
    return runtime, history, model#train using optimal learning rate and no. of epochs
runtime, history, model = train_model(train_dataset)

Epoch 1/30
    249/Unknown 26s 66ms/step - loss: 0.4708 - mae: 0.7408 - mape: 7.6897

C:\Users\jason\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


249/249 ━━━━━━━━━━━━━━━━━━━━ 29s 78ms/step - loss: 0.4695 - mae: 0.7391 - mape: 7.6724 - val_loss: 0.0112 - val_mae: 0.1112 - val_mape: 1.1847
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 17s 70ms/step - loss: 0.0148 - mae: 0.1341 - mape: 1.3934 - val_loss: 0.0270 - val_mae: 0.2077 - val_mape: 2.3208
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - loss: 0.0128 - mae: 0.1281 - mape: 1.3240 - val_loss: 0.0066 - val_mae: 0.0855 - val_mape: 0.8983
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 21s 83ms/step - loss: 0.0102 - mae: 0.1119 - mape: 1.1599 - val_loss: 0.0035 - val_mae: 0.0603 - val_mape: 0.6382
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 18s 71ms/step - loss: 0.0110 - mae: 0.1126 - mape: 1.1663 - val_loss: 0.0036 - val_mae: 0.0639 - val_mape: 0.6771
Epoch 6/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 19s 77ms/step - loss: 0.0058 - mae: 0.0845 - mape: 0.8775 - val_loss: 0.0031 - val_mae: 0.0595 - val_mape: 0.6284
Epoch 7/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 19s 78ms/step - loss: 0.0071 - mae: 0.0934 - ma

In [ ]:
# Visualize the loss using this cell
epochs = range(len(history.history['loss']))


plt.plot  ( epochs, history.history['mape'], label = 'mape')
# plt.plot  ( epochs, history.history['val_loss'], label = 'Validation')
plt.title ('Training and validation loss and mape')
plt.xlabel('epochs')
plt.legend();
#plt.ylim([0,0.01])

In [ ]:
assert runtime <= 1500, "Your model takes too long to train"
evaluations = model.evaluate(test_dataset)
mydict = {a:b for a,b in zip(model.metrics_names, evaluations)}
assert mydict['mape']!=None,"Please, include Mean Absolute Percentage Error (mape) as one of your metrics" #checks if accuracy included in metrics
assert mydict['mape']<0.5, "Your model needs to have evaluation MAPE of at most 0.5% and actual prediction MAPE of at most 10%. Try expanding your model, changing learning rate or training for more epochs." #checks if accuracy greater than 80%


## EVALUATE
Use the optimised model to predict and plot the validation data. Stop the loop before reaching the beginning of the last window, otherwise, your code will run out of data.

In [ ]:
forecast = model.predict(test_dataset)
# Plot your results alongside ground truth
plt.figure(figsize=(10, 6))
plt.plot(time_valid[window_size:],x_valid[window_size:], label='data')
plt.plot(time_valid[window_size:],forecast, label='RNN prediction on validation data')
plt.xlabel('time step')
plt.ylabel('label')
plt.title('RNN prediction')
plt.legend()
plt.ylim([9,9.5])
plt.xlim([0.145,0.15])

In [14]:
print('MAPE= ',np.mean(tf.keras.metrics.mean_absolute_percentage_error(x_valid[window_size:], np.array(forecast)).numpy()))
assert np.mean(tf.keras.metrics.mean_absolute_percentage_error(x_valid[window_size:], np.array(forecast)).numpy())<=10, "Your error is too large. Try improving your model or train for longer"

AttributeError: module 'keras._tf_keras.keras.metrics' has no attribute 'mean_absolute_percentage_error'